# QuantumFold-Advantage Benchmarking Suite
This notebook demonstrates the benchmarking of QuantumFold-Advantage against AlphaFold3 and Boltz2, including real structural metrics and 3D confidence-colored visualizations.

## 1. Environment Setup

In [ ]:
# Clone the repository and install dependencies
!git clone https://github.com/Tommaso-R-Marena/QuantumFold-Benchmark.git
%cd QuantumFold-Benchmark
!pip install -r requirements.txt
!pip install py3Dmol tabulate plotly ipywidgets

import sys
import os
import plotly.express as px
import plotly.graph_objects as go
import ipywidgets as widgets
from IPython.display import display, Markdown

sys.path.append(os.path.abspath("src"))

## 2. Run Rigorous Benchmark
We execute the full suite on CASP15, Miniproteins, and IDRs.

In [ ]:
!python scripts/run_benchmarks.py

## 3. Results Analysis

In [ ]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from qf_bench.metrics.statistics import compare_models

df = pd.read_csv('results/benchmark_results.csv')

# 1. Interactive RMSD Boxplot
fig_rmsd = px.box(df, x="dataset", y="rmsd", color="model", points="all",
                  title="RMSD Comparison across Datasets (Lower is Better)",
                  labels={"rmsd": "RMSD (Å)", "dataset": "Dataset", "model": "Model"})
fig_rmsd.update_layout(template="plotly_white")
fig_rmsd.show()

# 2. Interactive TM-score Boxplot
fig_tm = px.box(df, x="dataset", y="tm_score", color="model", points="all",
                 title="TM-score Comparison across Datasets (Higher is Better)",
                 labels={"tm_score": "TM-score", "dataset": "Dataset", "model": "Model"})
fig_tm.update_layout(template="plotly_white")
fig_tm.show()

# 3. Interactive pLDDT Bar Chart
summary_df = df.groupby(['dataset', 'model'])['plddt'].mean().reset_index()
fig_plddt = px.bar(summary_df, x="dataset", y="plddt", color="model", barmode="group",
                   title="Mean Model Confidence (pLDDT)",
                   labels={"plddt": "pLDDT", "dataset": "Dataset", "model": "Model"})
fig_plddt.update_layout(template="plotly_white")
fig_plddt.show()

# Statistical Comparison vs. QuantumFold-Advantage
baseline = "QuantumFold-Advantage"
stats_df = compare_models(df, baseline_model=baseline)

if not stats_df.empty:
    display(Markdown(f"### Paired Statistical Comparison vs. {baseline}"))
    display(stats_df.style.background_gradient(subset=['p_value_corrected'], cmap='RdYlGn_r', vmin=0, vmax=0.05))
else:
    display(Markdown("**No statistical comparison available (need at least 2 models and shared targets).**"))

### Statistical Model Comparison
We use paired significance tests (t-test and Wilcoxon signed-rank test) to compare **QuantumFold-Advantage** with the classical baseline models across all shared targets. 
- **Mean Difference**: Average difference in metric value (e.g., RMSD improvement).
- **p-value (Corrected)**: Statistical significance after Benjamini-Hochberg correction. Values < 0.05 (green) indicate significant performance differences.
- **Cohen's d**: Effect size indicating the magnitude of the difference.

## 4. 3D Structural Visualization (AlphaFold-style)
Visualize predicted structures colored by pLDDT confidence (Blue > 90, Cyan 70-90, Yellow 50-70, Orange < 50).

In [ ]:
import py3Dmol

def view_structure(pdb_path, label=None):
    with open(pdb_path, 'r') as f:
        data = f.read()
    
    view = py3Dmol.view(width=400, height=400)
    view.addModel(data, 'pdb')
    
    # Standard AF pLDDT coloring (Blue > 90, Cyan 70-90, Yellow 50-70, Orange < 50)
    view.setStyle({'cartoon': {'colorscheme': {'prop': 'b', 'gradient': 'roygb', 'min': 50, 'max': 90}}})
    if label:
        view.addLabel(label, {'position': {'x': 0, 'y': 0, 'z': 0}, 'backgroundColor': 'white', 'fontColor': 'black'})
    view.zoomTo()
    return view

def view_superimposed(ref_path, pred_path):
    with open(ref_path, 'r') as f:
        ref_data = f.read()
    with open(pred_path, 'r') as f:
        pred_data = f.read()

    view = py3Dmol.view(width=800, height=600)
    view.addModel(ref_data, 'pdb')
    view.addModel(pred_data, 'pdb')

    # Color Reference in grey, Prediction by pLDDT
    view.setStyle({'model': 0}, {'cartoon': {'color': 'grey', 'opacity': 0.5}})
    view.setStyle({'model': 1}, {'cartoon': {'colorscheme': {'prop': 'b', 'gradient': 'roygb', 'min': 50, 'max': 90}}})
    
    view.zoomTo()
    return view

def display_plddt_legend():
    display(Markdown("""
### pLDDT Confidence Legend
| Color | Confidence | pLDDT Range |
| :--- | :--- | :--- |
| <span style='color:blue'>Blue</span> | Very High | > 90 |
| <span style='color:cyan'>Cyan</span> | High | 70 - 90 |
| <span style='color:yellow'>Yellow</span> | Low | 50 - 70 |
| <span style='color:orange'>Orange</span> | Very Low | < 50 |
"""))

## 5. Interactive Structure Selection
Choose a target and a model to visualize its predicted structure, or compare it with the ground truth.

In [ ]:
def update_viz(target_id, model_name, compare):
    pred_path = f'results/{target_id}_{model_name}.pdb'
    ref_path = f'data/cache/{target_id}.pdb'
    
    display_plddt_legend()

    if compare:
        if os.path.exists(ref_path):
            print(f"Superimposing {model_name} prediction on ground truth for {target_id} (GT in grey)")
            view = view_superimposed(ref_path, pred_path)
            view.show()
        else:
            print(f"Ground truth for {target_id} not found in cache.")
            view_structure(pred_path, label=model_name).show()
    else:
        view_structure(pred_path, label=model_name).show()

target_dropdown = widgets.Dropdown(options=df['target_id'].unique(), description='Target:')
model_dropdown = widgets.Dropdown(options=df['model'].unique(), description='Model:')
compare_checkbox = widgets.Checkbox(value=False, description='Superimpose GT')

widgets.interactive(update_viz, target_id=target_dropdown, model_name=model_dropdown, compare=compare_checkbox)